In [85]:
import os
import time

spark_version = 'spark-3.5.4'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Fetched 6,555 B in 1s (4,894 B/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [86]:
# Import packages
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [87]:
# 1. Read the home_sales_revised.csv from the provided AWS S3 bucket location into a PySpark DataFrame.
# (1)
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)

df = spark.read.csv(SparkFiles.get('home_sales_revised.csv'), sep=',', header=True)

In [88]:
df.show(1)

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
only showing top 1 row



In [89]:
#creating unique column with year value, isolated from 'date' column in original df
# (2)
from pyspark.sql.functions import year

df = df.withColumn("sale_year", year(df['date']))

df.show(1)

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+---------+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|sale_year|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+---------+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|     2022|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+---------+
only showing top 1 row



In [90]:
# 2. Create a temporary view of the DataFrame.
# (3)
df.createOrReplaceTempView('df_temp_view')

In [91]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
# (3)
query1 = """
WITH t1 AS(
  SELECT sale_year, price
  FROM df_temp_view
  WHERE bedrooms = 4
)
SELECT sale_year AS Sale_Year, ROUND(AVG(price),2) AS AVG_Price
FROM t1
GROUP BY sale_year
ORDER BY sale_year asc;
"""

spark.sql(query1).show()

+---------+---------+
|Sale_Year|AVG_Price|
+---------+---------+
|     2019| 300263.7|
|     2020|298353.78|
|     2021|301819.44|
|     2022|296363.88|
+---------+---------+



In [92]:
# 4. What is the average price of a home for each year the home was built, that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
# (3)
query2 = """
WITH t1 AS(
  SELECT date_built, price
  FROM df_temp_view
  WHERE bedrooms = 3 AND bathrooms = 3
)
SELECT date_built AS Year_Built, ROUND(AVG(price),2) AS AVG_Price
FROM t1
GROUP BY date_built
ORDER BY date_built asc;
"""

spark.sql(query2).show()

+----------+---------+
|Year_Built|AVG_Price|
+----------+---------+
|      2010|292859.62|
|      2011|291117.47|
|      2012|293683.19|
|      2013|295962.27|
|      2014|290852.27|
|      2015| 288770.3|
|      2016|290555.07|
|      2017|292676.79|
+----------+---------+



In [93]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms,
# with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
query3 = """
WITH t1 AS(
  SELECT date_built, price
  FROM df_temp_view
  WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
)
SELECT date_built AS Year_Built, ROUND(AVG(price),2) AS AVG_Price
FROM t1
GROUP BY date_built
ORDER BY date_built asc;
"""

spark.sql(query3).show()

+----------+---------+
|Year_Built|AVG_Price|
+----------+---------+
|      2010|285010.22|
|      2011|276553.81|
|      2012|307539.97|
|      2013|303676.79|
|      2014|298264.72|
|      2015|297609.97|
|      2016| 293965.1|
|      2017|280317.58|
+----------+---------+



In [94]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

query4 = """
with t1 AS (
  SELECT view, price
  FROM df_temp_view
  WHERE price >= 350000
)
SELECT view as Views, ROUND(AVG(price),2) AS AVG_Price
FROM t1
GROUP BY view
ORDER BY view desc;
"""

spark.sql(query4).show()

print("--- %s seconds ---" % (time.time() - start_time))

+-----+----------+
|Views| AVG_Price|
+-----+----------+
|   99|1061201.42|
|   98|1053739.33|
|   97|1129040.15|
|   96|1017815.92|
|   95| 1054325.6|
|   94| 1033536.2|
|   93|1026006.06|
|   92| 970402.55|
|   91|1137372.73|
|   90|1062654.16|
|    9| 401393.34|
|   89|1107839.15|
|   88|1031719.35|
|   87| 1072285.2|
|   86|1070444.25|
|   85|1056336.74|
|   84|1117233.13|
|   83|1033965.93|
|   82| 1063498.0|
|   81|1053472.79|
+-----+----------+
only showing top 20 rows

--- 0.9418189525604248 seconds ---


In [95]:
# 7. Cache the the temporary table home_sales.
# (3)
spark.sql("cache table df_temp_view")

DataFrame[]

In [96]:
# 8. Check if the table is cached.
spark.catalog.isCached('df_temp_view')

True

In [97]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

query5 = """
with t1 AS (
  SELECT view, price
  FROM df_temp_view
  WHERE price >= 350000
)
SELECT view as Views, ROUND(AVG(price),2) AS AVG_Price
FROM t1
GROUP BY view
ORDER BY view desc;
"""

spark.sql(query5).show()



print("--- %s seconds ---" % (time.time() - start_time))


+-----+----------+
|Views| AVG_Price|
+-----+----------+
|   99|1061201.42|
|   98|1053739.33|
|   97|1129040.15|
|   96|1017815.92|
|   95| 1054325.6|
|   94| 1033536.2|
|   93|1026006.06|
|   92| 970402.55|
|   91|1137372.73|
|   90|1062654.16|
|    9| 401393.34|
|   89|1107839.15|
|   88|1031719.35|
|   87| 1072285.2|
|   86|1070444.25|
|   85|1056336.74|
|   84|1117233.13|
|   83|1033965.93|
|   82| 1063498.0|
|   81|1053472.79|
+-----+----------+
only showing top 20 rows

--- 0.9367773532867432 seconds ---


In [98]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
# (2)
df.write.partitionBy("date_built").mode("overwrite").parquet('date_built_partitioned_parquet')


In [99]:
# 11. Read the parquet formatted data.
# (2)
partitioned_parquet_df = spark.read.parquet('date_built_partitioned_parquet')

In [100]:
# 12. Create a temporary table for the parquet data.
partitioned_parquet_df.createOrReplaceTempView('pqd_tempView')

In [101]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

query6 = """
with t1 AS (
  SELECT view, price
  FROM pqd_tempView
  WHERE price >= 350000
)
SELECT view as Views, ROUND(AVG(price),2) AS AVG_Price
FROM t1
GROUP BY view
ORDER BY view desc;
"""

spark.sql(query6).show()


print("--- %s seconds ---" % (time.time() - start_time))

+-----+----------+
|Views| AVG_Price|
+-----+----------+
|   99|1061201.42|
|   98|1053739.33|
|   97|1129040.15|
|   96|1017815.92|
|   95| 1054325.6|
|   94| 1033536.2|
|   93|1026006.06|
|   92| 970402.55|
|   91|1137372.73|
|   90|1062654.16|
|    9| 401393.34|
|   89|1107839.15|
|   88|1031719.35|
|   87| 1072285.2|
|   86|1070444.25|
|   85|1056336.74|
|   84|1117233.13|
|   83|1033965.93|
|   82| 1063498.0|
|   81|1053472.79|
+-----+----------+
only showing top 20 rows

--- 0.9123907089233398 seconds ---


In [102]:
# 14. Uncache the home_sales temporary table.
# (3)
spark.sql("uncache table df_temp_view")

DataFrame[]

In [103]:
# 15. Check if the home_sales is no longer cached
# (3)
spark.catalog.isCached('df_temp_view')

False